<a href="https://colab.research.google.com/github/marags-web/marags-web/blob/main/IDS/IDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Network intrusion detection and classification

### 1. Import dependencies

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, Dropout
from keras.models import Sequential
from google.colab import drive

In [3]:
#drive.mount('/content/drive/') 
#path = '/content/drive/My Drive/Colab Notebooks/datasets/network_data_train.txt'
url =  "https://raw.githubusercontent.com/marags-web/marags-web/main/IDS/data/network_data_train.txt"


### 2. Data Preprocessing

In [4]:
# data does not have column headers, so define them
headers = ["duration","protocol_type","service","flag","src_bytes",
           "dst_bytes","land","wrong_fragment","urgent","hot",
           "num_failed_logins","logged_in","num_compromised","root_shell",
           "su_attempted","num_root","num_file_creations","num_shells",
           "num_access_files","num_outbound_cmds","is_host_login",
           "is_guest_login","count","srv_count","serror_rate", 
           "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate", 
           "diff_srv_rate", "srv_diff_host_rate","dst_host_count",
           "dst_host_srv_count","dst_host_same_srv_rate",
           "dst_host_diff_srv_rate","dst_host_same_src_port_rate",
           "dst_host_srv_diff_host_rate","dst_host_serror_rate",
           "dst_host_srv_serror_rate","dst_host_rerror_rate",
           "dst_host_srv_rerror_rate","attack", "last_flag"]

# read csv
data = pd.read_csv(url, names=headers)

# choose relevent features
features = ['duration','src_bytes','dst_bytes','num_file_creations', 
           'num_shells','num_failed_logins','wrong_fragment', 
           'urgent', 'is_guest_login', 'su_attempted', 'land', 'attack']
df = data[features]

In [5]:
pd.set_option('display.max_columns', None)
print(df.head())

   duration  src_bytes  dst_bytes  num_file_creations  num_shells  \
0         0        491          0                   0           0   
1         0        146          0                   0           0   
2         0          0          0                   0           0   
3         0        232       8153                   0           0   
4         0        199        420                   0           0   

   num_failed_logins  wrong_fragment  urgent  is_guest_login  su_attempted  \
0                  0               0       0               0             0   
1                  0               0       0               0             0   
2                  0               0       0               0             0   
3                  0               0       0               0             0   
4                  0               0       0               0             0   

   land   attack  
0     0   normal  
1     0   normal  
2     0  neptune  
3     0   normal  
4     0   normal  


In [6]:
class_labels = ['normal', 'dos', 'probing', 'u2r', 'r2l']

# data has too many classes
# to simplify problem, categorize them generic network attack types : Probing, U2R, R2L, and DOS

dos = ['neptune', 'apache2', 'processtable', 'smurf', 'back', 'snmpguess', 'mailbomb', 'snmpgetattack',
              'pod', 'multihop', 'teardrop', 'sqlattack', 'land']
probing = ['mscan', 'satan', 'saint', 'portsweep', 'ipsweep', 'nmap', 'spy']
u2r = ['buffer_overflow', 'ps', 'xterm', 'perl', 'loadmodule', 'imap']
r2l = ['guess_passwd', 'warezmaster', 'httptunnel', 'named', 'sendmail', 'xlock','xsnoop', 'rootkit', 
       'ftp_write', 'worm', 'phf', 'udpstorm', 'warezclient']

# replace all values in dataframe with corresponding int value

df['attack'].replace('normal', 0, inplace=True)
df['attack'].replace(dos, 1, inplace=True)
df['attack'].replace(probing, 2, inplace=True)
df['attack'].replace(u2r, 3, inplace=True)
df['attack'].replace(r2l, 4, inplace=True)

# shuffle the data
df = df.sample(frac=1).reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [7]:
df['attack'].value_counts()

0    67343
1    45934
2    11658
4      985
3       53
Name: attack, dtype: int64

In [8]:
# split data into inputs/outputs
features = df[['duration','src_bytes','dst_bytes','num_file_creations', 
           'num_shells','num_failed_logins','wrong_fragment', 
           'urgent', 'is_guest_login', 'su_attempted', 'land']]
labels = df['attack']

X = np.array(features)
y = np.array(labels).reshape(-1, 1)

# make sure both arrays have correct dimensions
print(X.shape)
print(y.shape)

(125973, 11)
(125973, 1)


### 3. Buidling the model

In [9]:
model = Sequential()

model.add(Dense(units=32, activation='relu', input_dim=(11)))

model.add(Dense(units=24, activation='relu'))
model.add(Dropout(.2))

model.add(Dense(units=11, activation='relu'))
model.add(Dropout(.2))

model.add(Dense(units=8, activation='relu'))

model.add(Dense(units=5, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer="adam", metrics=['accuracy'],run_eagerly=True)

In [10]:
model.fit(X, y, epochs=10, batch_size=32)

Epoch 1/10
3937/3937 [==============================] - 72s 18ms/step - loss: 425.6548 - accuracy: 0.6809
Epoch 2/10
3937/3937 [==============================] - 70s 18ms/step - loss: 11.7928 - accuracy: 0.8070
Epoch 3/10
3937/3937 [==============================] - 71s 18ms/step - loss: 9.3976 - accuracy: 0.8276
Epoch 4/10
3937/3937 [==============================] - 70s 18ms/step - loss: 2.4005 - accuracy: 0.8562
Epoch 5/10
3937/3937 [==============================] - 69s 18ms/step - loss: 0.7760 - accuracy: 0.8666
Epoch 6/10
3937/3937 [==============================] - 71s 18ms/step - loss: 0.6564 - accuracy: 0.8698
Epoch 7/10
3937/3937 [==============================] - 67s 17ms/step - loss: 0.4742 - accuracy: 0.8721
Epoch 8/10
3937/3937 [==============================] - 68s 17ms/step - loss: 0.4581 - accuracy: 0.8724
Epoch 9/10
3937/3937 [==============================] - 66s 17ms/step - loss: 0.4198 - accuracy: 0.8731
Epoch 10/10
3937/3937 [==============================] - 67s 

In [11]:
test_path = 'url =  "https://raw.githubusercontent.com/marags-web/marags-web/main/IDS/data/network_data_test.txt'

test = pd.read_csv(url, names=headers)

features = ['duration','src_bytes','dst_bytes','num_file_creations', 
           'num_shells','num_failed_logins','wrong_fragment', 
           'urgent', 'is_guest_login', 'su_attempted', 'land', 'attack']

test = test[features]

test['attack'].replace('normal', 0, inplace=True)
test['attack'].replace(dos, 1, inplace=True)
test['attack'].replace(probing, 2, inplace=True)
test['attack'].replace(u2r, 3, inplace=True)
test['attack'].replace(r2l, 4, inplace=True)

X_test, y_test = df.drop('attack', axis=1), df['attack']

val_loss, val_acc = model.evaluate(X_test, y_test) 

3937/3937 [==============================] - 39s 10ms/step - loss: 0.3988 - accuracy: 0.8865


In [12]:
print(val_acc)
print(val_loss)

0.8865153789520264
0.39880964159965515


#### 88% Accuracy on validation data

In [13]:
# generate a random index to make a prediction on
import random
prediction_index = random.randint(0, len(X_test))

# make prediction
pred_input = np.array(list(X_test.iloc[prediction_index])).reshape(1, 11)
prediction = class_labels[model.predict( pred_input ).argmax()]
actual = class_labels[y_test.iloc[prediction_index]]

# compare prediction vs actual value
print(f'Predicted Value: {prediction}')
print(f'Actual Value: {actual}')

1/1 [==============================] - 0s 40ms/step
Predicted Value: normal
Actual Value: normal
